# **Trabalho Final da disciplina de Processamento de Linguagem Natural**
### Ana Luísa Araújo Bastos - 2021031459

### Considerações: devido a limitações do meu computador e ao limite imposto pelo Colab, o trabalho foi executado em um servidor e os dados gerados carregados aqui.

# Predição de Interação entre Proteínas utilizando NLP

## Objetivo:
O objetivo deste projeto é construir um modelo capaz de prever se duas proteínas irão interagir ou não com base em suas sequências. Esse tipo de predição é essencial para entender redes biológicas, interações celulares e o desenvolvimento de terapias.

## Preparando o ambiente

In [26]:
# Instalação das bibliotecas necessárias
!pip install --upgrade transformers datasets evaluate
!pip install nltk sklearn pandas biopython seaborn matplotlib requests tqdm


  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [31]:
# Importação das bibliotecas necessárias
import torch
import torch.optim as optim
import torch.nn as nn
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import tarfile
import urllib.request
import os
from  collections import Counter
from datasets import load_dataset
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from tqdm import tqdm
import gzip
!pip install biopython
import Bio
from Bio import SeqIO
import json
import numpy as np
import joblib
from tqdm import tqdm
import random
import networkx as nx
from PIL import Image



## Preparando os dados

Os dados uilizados foram retirados do **STRING Database**.
O STRING é uma base de dados amplamente utilizada, que contém informações sobre interações proteína-proteína (PPI) e fornece um formato que pode ser utilizado diretamente para treinamentos de modelos de aprendizado de máquina.

Os arquivos foram baixados e disponibilizados no meu GitHub.
Para esse trabalho, selecionei apenas dados com uma confiança acima de 40% e de proteínas humanas.

### Download

In [28]:
# Baixar os arquivos
def download_file(url, output_path):
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    with open(output_path, 'wb') as file, tqdm(
        desc=f"Downloading {url.split('/')[-1]}",
        total=total_size,
        unit='B',
        unit_scale=True,
        unit_divisor=1024,
    ) as bar:
        for data in response.iter_content(chunk_size=1024):
            file.write(data)
            bar.update(len(data))

# Criar diretório para os dados
os.makedirs("dados", exist_ok=True)

# URLs dos arquivos

ppi_url =  "https://github.com/AnaLuisaAB/TP_FINAL-NLP/raw/main/dados/9606.protein.links.v12.0.min400.onlyAB.txt.gz"
sequences_url ="https://github.com/AnaLuisaAB/TP_FINAL-NLP/raw/main/dados/9606.protein.sequences.v12.0.fa.gz"

# Baixar os arquivos
download_file(ppi_url, "dados/ppi_data.txt.gz")
download_file(sequences_url, "dados/sequences.fa.gz")


### Pré-processamento




Primeiro extraí os arquivos e filtrei apenas interações com ```combined_score```acima de 500 para melhorar mais a confiança.
Depois, para adquirir as sequências das proteínas envolvidas na interação, combinei o arquivo original do dataset com o arquivos FASTA das sequências.

Além do citado acima, possíveis pares sem informação foram removidos e foi feita uma redução do dataset, já que o original tem bilhões de interações.

Esse número variou experimentalmente e acabou em 75.000 amostras positivas (após gerar as negativas, ficaram 150.000 no total). Depois, para novos experimentos, mudei para 200.000 no total (100.000 positivas e 100.000 negativas)

Além disso, foi gerado um conjunto de amostra com 2500 pares de sequências que não inclui s sequências utilizadas  (após gerar as negativas, ficaram 5000 no total).


In [29]:
# Função para carregar interações PPI
def process_ppi(file_path, threshold = 500):
    """
    Processa o arquivo de interações PPI e retorna os pares de proteínas com maior confiança.
    :param file_path: Caminho para o arquivo PPI (links.detailed).
    :param threshold: Pontuação mínima para considerar uma interação (default = 500).
    """
    ppi_data = pd.read_csv(file_path, sep=" ", compression='gzip')
    ppi_data = ppi_data[ppi_data["combined_score"] >= threshold]
    return ppi_data[["protein1", "protein2", "combined_score"]]

# Função para carregar sequências de proteínas
def process_sequences(file_path):
    """
    Processa o arquivo de sequências FASTA e retorna um dicionário {protein_id: sequence}.
    :param file_path: Caminho para o arquivo de sequências (sequences.fa).
    """
    sequences = {}
    with gzip.open(file_path, "rt") as f:
        for record in SeqIO.parse(f, "fasta"):
            sequences[record.id] = str(record.seq)
    return sequences




In [22]:
# Caminhos dos arquivos
ppi_path = "dados/ppi_data.txt.gz"
sequences_path = "dados/sequences.fa.gz"
# Processar os arquivos
ppi_data = process_ppi(ppi_path)
sequences_dict = process_sequences(sequences_path)

# Adicionar sequências às interações PPI
ppi_data["Protein1_Seq"] = ppi_data["protein1"].map(sequences_dict)
ppi_data["Protein2_Seq"] = ppi_data["protein2"].map(sequences_dict)

# Remover pares sem sequências
ppi_data = ppi_data.dropna()

# Reduzir o dataset para 75.000 amostras aleatórias
ppi_data_reduced = ppi_data.sample(n=75000, random_state=42)

# Visualizar as primeiras linhas
print(ppi_data_reduced.head())
print(ppi_data_reduced.info())
print(ppi_data_reduced.describe())

# Gerar um dataset de amostra reduzido (2500 amostras) que não inclui as amostras usadas
ppi_data_without_used = ppi_data.drop(ppi_data_reduced.index)
ppi_data_sample = ppi_data_without_used.sample(n=2500, random_state=42)


                    protein1              protein2  combined_score  \
906054  9606.ENSP00000451812  9606.ENSP00000483236             558   
695634  9606.ENSP00000353741  9606.ENSP00000394496             518   
58533   9606.ENSP00000219334  9606.ENSP00000262094             575   
196183  9606.ENSP00000255266  9606.ENSP00000262340             813   
616442  9606.ENSP00000336661  9606.ENSP00000344465             566   

                                             Protein1_Seq  \
906054  MGWKMASPTDGTDLEASLLSFEKLDRASPDLWPEQLPGVAEFAASF...   
695634  MADDPSAADRNVEIWKIKKLIKSLEAARGNGTSMISLIIPPKDQIS...   
58533   MVLYTTPFPNSCLSALHCVSWALIFPCYWLVDRLAASFIPTTYEKR...   
196183  MGEVTAEEVEKFLDSNIGFAKQYYNLHYRAKLISDLLGAKEAAVDF...   
616442  MSSEQSAPGASPRAPRPGTQKSSGAVTKKGERAAKEKPATVLPPVG...   

                                             Protein2_Seq  
906054  MSLHGKRKEIYKYEAPWTVYAMNWSVRPDKRFRLALGSFVEEYNNK...  
695634  MYGPGSQLGKSGNNSWAKERGCSIACQGSLTSARLHAPSIGERPLS...  
58533   MTDGDYDYLIKLLALGDSGVGKTT

### Gerar amostras negativas
Como o dataset só tem representações de proteínas que interagem, foi necessário gerar as amostras negativas. Para isso, enxerguei todas as proteínas como nós de um grafo que possui arestas entre as proteínas que interagem entre si. Dessa forma, basta gerar pares que não possuem arestas.



1.   Construção do Grafo: Usamos o dataset de interações para criar um grafo de interações.
2.   Gerar Pares Negativos: Geramos pares de proteínas aleatórias que não estão conectadas no grafo.
3.   Combinação de Exemplos: Combinamos as interações positivas (do dataset original) e as interações negativas (geradas a partir do grafo) para criar um dataset balanceado.

In [30]:
# Criar o grafo de interações usando a amostra reduzida
G = nx.Graph()

# Adicionar as interações como arestas no grafo
for _, row in ppi_data_reduced.iterrows():
    G.add_edge(row['protein1'], row['protein2'])

# Verificar o número de nós e arestas no grafo
print(f"Número de nós (proteínas): {len(G.nodes())}")
print(f"Número de arestas (interações): {len(G.edges())}")

Número de nós (proteínas): 16027
Número de arestas (interações): 75000


In [24]:
proteins = list(G.nodes())
num_negatives = len(ppi_data_reduced)  # O mesmo número de pares negativos que o número de pares positivos
negative_pairs = []
while len(negative_pairs) < num_negatives:
    p1, p2 = random.sample(proteins, 2)  # Seleciona duas proteínas aleatórias

    # Verificar se as duas proteínas não estão conectadas no grafo
    if not G.has_edge(p1, p2):
        negative_pairs.append((p1, p2))


### Combinar datasets
Agora precisamos unir o dataset positivo (possui interação) que já tinhamos, com o negativo (não possui interação) que acabamos de criar.
Além disso, o datasset postivo antes tinha valores de ```combined_score``` entre 500 e 1000, então passamos a utilizar apenas 1 para indicar interação e 0 caso contrário.



In [25]:
def create_balanced_dataset(ppi_data_reduced, sequences_dict):
    """
    Cria um dataset balanceado com pares positivos e negativos de interações proteína-proteína.

    Parâmetros:
    - ppi_data_reduced: DataFrame com as interações positivas (colunas: protein1, protein2, combined_score, Protein1_Seq, Protein2_Seq).
    - sequences_dict: Dicionário com as sequências de proteínas (chave: protein_id, valor: sequência).

    Retorna:
    - combined_data: DataFrame balanceado com pares positivos e negativos.
    """
    # Criar o grafo de interações usando a amostra reduzida
    G = nx.Graph()

    # Adicionar as interações como arestas no grafo
    for _, row in ppi_data_reduced.iterrows():
        G.add_edge(row['protein1'], row['protein2'])

    # Verificar o número de nós e arestas no grafo
    print(f"Número de nós (proteínas): {len(G.nodes())}")
    print(f"Número de arestas (interações): {len(G.edges())}")

    # Gerar pares negativos
    proteins = list(G.nodes())
    num_negatives = len(ppi_data_reduced)  # O mesmo número de pares negativos que o número de pares positivos
    negative_pairs = []
    while len(negative_pairs) < num_negatives:
        p1, p2 = random.sample(proteins, 2)  # Seleciona duas proteínas aleatórias

        # Verificar se as duas proteínas não estão conectadas no grafo
        if not G.has_edge(p1, p2):
            negative_pairs.append((p1, p2))

    # Criar rótulos para os pares
    negative_labels = np.zeros(len(negative_pairs))  # Todos os pares negativos não têm interação = 0

    # Criar um dataframe para os exemplos negativos
    negative_data = pd.DataFrame(negative_pairs, columns=['protein1', 'protein2'])
    negative_data['combined_score'] = 0  # Pontuação irrelevante para exemplos negativos

    # Adicionar as sequências de proteínas para os pares negativos
    negative_data['Protein1_Seq'] = negative_data['protein1'].map(sequences_dict)
    negative_data['Protein2_Seq'] = negative_data['protein2'].map(sequences_dict)

    # Combinar os dados positivos e negativos
    positive_data = ppi_data_reduced[['protein1', 'protein2', 'combined_score', 'Protein1_Seq', 'Protein2_Seq']]
    positive_data['combined_score'] = 1  # Atribuir 1 para todos os pares com interação

    combined_data = pd.concat([positive_data, negative_data[['protein1', 'protein2', 'combined_score', 'Protein1_Seq', 'Protein2_Seq']]], ignore_index=True)

    # Exibir informações do dataset
    print(f"Dataset combinado com {len(combined_data)} entradas.")
    print("-------------NEGATIVE-------------")
    print(negative_data.head())
    print("\n-------------POSITIVE-------------")
    print(positive_data.head())
    print("\n-------------COMBINED-------------")
    print(combined_data.info())
    print(combined_data.describe())
    print(combined_data.head())


    return combined_data

# Balancear o dataset de amostra
ppi_data_sample = create_balanced_dataset(ppi_data_sample, sequences_dict)

# Gerar o dataset que será usado com balanceamento
combined_data = create_balanced_dataset(ppi_data_reduced, sequences_dict)


# Juntar a sequência dos pares de proteína
combined_data["combined_sequence"] = combined_data["Protein1_Seq"] + combined_data["Protein2_Seq"]
ppi_data_sample["combined_sequence"] = ppi_data_sample["Protein1_Seq"] + combined_data["Protein2_Seq"]

# Salvar o dataset final (reaproveitamento quando não há mudança no número de amostras)
combined_data.to_csv("dados/processed_ppi_data.csv", index=False)

# Salvar o dataset de amostra
ppi_data_sample.to_csv("dados/processed_ppi_data_sample.csv", index=False)

Número de nós (proteínas): 3677
Número de arestas (interações): 2500
Dataset combinado com 5000 entradas.
-------------NEGATIVE-------------
               protein1              protein2  combined_score  \
0  9606.ENSP00000359765  9606.ENSP00000492893               0   
1  9606.ENSP00000357459  9606.ENSP00000367123               0   
2  9606.ENSP00000225916  9606.ENSP00000358423               0   
3  9606.ENSP00000265970  9606.ENSP00000251775               0   
4  9606.ENSP00000429084  9606.ENSP00000330393               0   

                                        Protein1_Seq  \
0  MVSSGCRMRSLWFIIVISFLPNTEGFSRAALPFGLVRRELSCEGYS...   
1  MNPRQGYSLSGYYTHPFQGYEHRQLRYQQPGPGSSPSSFLLKQIEF...   
2  MAEPSQAPTPAPAAQPRPLQSPAPAPTPTPAPSPASAPIPTPTPAP...   
3  MAQISSNSGFKECPSSHPEPTRAKDVDKEEALQMEAEALAKLQKDR...   
4  MTSIHFVVHPLPGTEDQLNDRLREVSEKLNKYNLNSHPPLNVLEQA...   

                                        Protein2_Seq  
0  MVNLTSMSGFLLMGFSDERKLQILHALVFLVTYLLALTGNLLIITI...  
1  MELQPPEASIAVVSIPRQ

## Aplicação e Avaliação dos algoritmos

### Geração de embeddings

O primeiro passo foi escolher o modelo para gerar os embeddings. Inicialmente pensei no ProtBert, uma versão do Bert fine-tunado para sequências de proteínas, porém, após uma série de testes, descobri que ele estava gerando embeddings iguais, de modo que todas as classificações iam para uma classe, mas não consegui identificar o motivo, por isso, resolvi mudar o modelo.

Optei pelo ESM-2 (Evolutionary Scale Modeling), que também é treinado para aminoácidos, na versão `esm2_t6_8M_UR50D` que é mais leve.

Os embeddings foram gerados com tamanho máximo de 1024.

In [9]:
# Configuração do modelo
MODEL_NAME = "facebook/esm2_t6_8M_UR50D"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME).to(device)

# Função para gerar embeddings
def generate_embeddings(sequences, tokenizer, model, device, batch_size=256):
    all_embeddings = []
    for i in tqdm(range(0, len(sequences), batch_size), desc="Gerando embeddings"):
        batch = list(sequences[i : i + batch_size])
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
        with torch.no_grad():
            outputs = model(**inputs).last_hidden_state.mean(dim=1)
        all_embeddings.extend(outputs.cpu().numpy())
    return np.vstack(all_embeddings)



'''combined_data = pd.read_csv("dados/combined_data.csv")''' # Descomentar em caso de reutilização

# Gerar embeddings
embeddings_path = "dados/embeddings.npy"
labels_path = "dados/labels.npy"

embeddings = generate_embeddings(combined_data["combined_sequence"].values, tokenizer, model, device)
labels = combined_data["combined_score"].values

np.save(embeddings_path, embeddings)
np.save(labels_path, labels)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/95.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/93.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/775 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/31.4M [00:00<?, ?B/s]

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


'# Gerar embeddings\nembeddings_path = "string_data/embeddings.npy"\nlabels_path = "string_data/labels.npy"\n\nembeddings = generate_embeddings(combined_data["combined_sequence"].values, tokenizer, model, device)\nlabels = combined_data["combined_score"].values\n\nnp.save(embeddings_path, embeddings)\nnp.save(labels_path, labels)'

### Classificação

Para classificação testei duas opções, o Random Forest e uma simulação de LSTM com atenção.

#### **Random Forest**

Foram testados vários valores de estimadores, começando em 50 e subindo de 50 em 50 até 300, como não houve muita diferença de 250 para 300, decidir parar em 300.

In [ ]:
# Divisão treino/teste
X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, test_size=0.2, random_state=42)

# Definição dos parâmetros
clf = RandomForestClassifier(n_estimators=300, random_state=42)
clf.fit(X_train, y_train)
joblib.dump(clf, 'dados/random_forest_model.pkl')

# Avaliação do modelo
y_pred = clf.predict(X_test)
for i in range(len(y_pred)):
    if y_pred[i] > 0.5:
        y_pred[i] = 1
    else:
        y_pred[i] = 0

np.save('dados/y_pred_test.npy', y_pred)

# Relatório de classificação
report = classification_report(y_test, y_pred, output_dict=True)
pd.DataFrame(report).transpose().to_csv("dados/classification_report_RF.csv")

# Matriz de confusão
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title("Matriz de Confusão")
plt.xlabel("Previsto")
plt.ylabel("Real")
plt.savefig("dados/confusion_matrix_RF.png")
plt.show()

# Métricas
metrics = {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred, average='binary'),
        'recall': recall_score(y_test, y_pred, average='binary'),
        'f1_score': f1_score(y_test, y_pred, average='binary')
    }
with open('dados/metricsRF300.json', 'w') as f:
    json.dump(metrics, f)
    print(metrics)



**Carregando resultados**

In [33]:
# Função para carregar e imprimir o relatório de classificação
def load_and_print_classification_report(report_path, model_name):
    print(f"\n=== Relatório de Classificação - {model_name} ===")
    report_df = pd.read_csv(report_path, index_col=0)
    print(report_df)

# Função para carregar e imprimir as métricas
def load_and_print_metrics(metrics_path, model_name):
    print(f"\n=== Métricas - {model_name} ===")
    with open(metrics_path, "r") as f:
        metrics = json.load(f)
    for key, value in metrics.items():
        print(f"{key}: {value:.4f}")
# Função para carregar e exibir uma imagem
def load_and_display_image(image_path, title):
    img = Image.open(image_path)
    plt.figure(figsize=(6, 6))
    plt.imshow(img)
    plt.title(title)
    plt.axis("off")
    plt.show()


In [34]:
# Caminhos dos arquivos
report_rf_path = "dados/classification_report_RF1.csv"
metrics_rf_path = "dados/metricsRF1.json"
#confusion_matrix_rf_path = "string_data/confusion_matrix_RF1.png"

# Carregar e imprimir os relatórios e métricas do Random Forest
load_and_print_classification_report(report_rf_path, "Random Forest")
load_and_print_metrics(metrics_rf_path, "Random Forest")
#load_and_display_image(confusion_matrix_rf_path, "Matriz de Confusão - Random Forest")


=== Relatório de Classificação - Random Forest ===
              precision    recall  f1-score       support
0.0            0.695572  0.730168  0.712451  15039.000000
1.0            0.714487  0.678765  0.696168  14961.000000
accuracy       0.704533  0.704533  0.704533      0.704533
macro avg      0.705030  0.704467  0.704309  30000.000000
weighted avg   0.705005  0.704533  0.704330  30000.000000

=== Métricas - Random Forest ===
accuracy: 0.7045
precision: 0.7145
recall: 0.6788
f1_score: 0.6962


#### **LSTM com atenção**

Assim como para o Random Forest, também testei várias combinações de hiperparâmetros.

Batch size: 64, 128, 256

Número de camadas: 2, 3

Dimensão da camada oculta: 128, 256, 512

Número de épocas: 20, 50, 100

Para os parâmetros testados, o melhor resultado foi para batch_size = 128, num_layers = 2, hidden_dim = 512 e num_epochs = 50 (o early stopping estava parando antes de chegar às 50).

In [ ]:
# Carregar embeddings e labels
embeddings = np.load("dados/embeddings.npy")
labels = np.load("dados/labels.npy")

# Divisão treino/teste
X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, test_size=0.2, random_state=42)

# Redimensionar os dados para o formato esperado pela LSTM (batch_size, sequence_length, input_dim)
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))  # (batch_size, 1, input_dim)
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))      # (batch_size, 1, input_dim)

# Converter para tensores do PyTorch
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

# Definir o modelo LSTM com Atenção Aprimorado
class LSTMAttention(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(LSTMAttention, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True, bidirectional=True, num_layers=2)
        self.attention = nn.Linear(hidden_dim * 2, 1)
        self.fc1 = nn.Linear(hidden_dim * 2, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, ou.tput_dim)
        self.dropout = nn.Dropout(0.5)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        attention_weights = torch.softmax(self.attention(lstm_out), dim=1)
        weighted_out = torch.sum(lstm_out * attention_weights, dim=1)
        x = torch.relu(self.fc1(weighted_out))
        x = self.dropout(x)
        x = self.fc2(x)
        return self.sigmoid(x)

# Configuração do modelo
input_dim = X_train.shape[2]  # Dimensão de entrada (tamanho do embedding)
hidden_dim = 512              # Dimensão oculta
output_dim = 1                # Saída binária (interação ou não)
model = LSTMAttention(input_dim, hidden_dim, output_dim)


# Otimizador com scheduler de taxa de aprendizado
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Treinamento do modelo
num_epochs = 10
batch_size = 128
best_val_loss = float('inf')
patience = 0

for epoch in range(num_epochs):
    model.train()
    for i in range(0, len(X_train), batch_size):
        inputs = X_train[i:i+batch_size]
        targets = y_train[i:i+batch_size]

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), targets)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

    # Avaliação no conjunto de validação
    model.eval()
    with torch.no_grad():
        y_pred_probs = model(X_test).squeeze().numpy()
        y_pred = (y_pred_probs > 0.5).astype(int)
        val_loss = criterion(torch.tensor(y_pred_probs), y_test).item()

    print(classification_report(y_test.numpy(), y_pred))
    print("Acurácia:", accuracy_score(y_test.numpy(), y_pred))

    # Early Stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience = 0
    else:
        patience += 1
        if patience >= 5:  # Parar após 5 épocas sem melhoria
            print("Early stopping")
            break

    scheduler.step()  # Atualizar a taxa de aprendizado

torch.save(model.state_dict(), "dados/lstm_attention_model.pth")
report = classification_report(y_test, y_pred, output_dict=True)
pd.DataFrame(report).transpose().to_csv("dados/classification_report_lstm_attention.csv")

# Matriz de confusão
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title("Matriz de Confusão (LSTM com Atenção)")
plt.xlabel("Previsto")
plt.ylabel("Real")
plt.savefig("dados/confusion_matrix_lstm_attention.png")
plt.show()

# Métricas
metrics = {
    'accuracy': accuracy_score(y_test, y_pred),
    'precision': precision_score(y_test, y_pred, average='binary'),
    'recall': recall_score(y_test, y_pred, average='binary'),
    'f1_score': f1_score(y_test, y_pred, average='binary')
}
with open('dados/metrics_lstm_attention.json', 'w') as f:
    json.dump(metrics, f)
    print(metrics)

In [35]:
# Caminhos dos arquivos
report_lstm_path = "dados/classification_report_lstm_attention1.csv"
metrics_lstm_path = "dados/metrics_lstm_attention1.json"
#confusion_matrix_lstm_path = "string_data/confusion_matrix_lstm_attention1.png"

# Carregar e imprimir os relatórios e métricas do Random Forest
load_and_print_classification_report(report_lstm_path, "LSTM com Atenção")
load_and_print_metrics(metrics_lstm_path, "LSTM com Atenção")
#load_and_display_image(confusion_matrix_lstm_path, "Matriz de Confusão - LSTM com Atenção")


=== Relatório de Classificação - LSTM com Atenção ===
              precision  recall  f1-score  support
class                                             
0.0                0.68    0.75      0.71    15039
1.0                0.72    0.65      0.68    14961
accuracy           0.70    0.70      0.70    30000
macro avg          0.70    0.70      0.70    30000
weighted avg       0.70    0.70      0.70    30000

=== Métricas - LSTM com Atenção ===
accuracy: 0.7000
precision: 0.7212
recall: 0.6495
f1_score: 0.6835


### Aumentando tamanho dos embeddings

Como não estava satisfeita com os aproximadamente 70% de acurácia e precisão, resolvi testar gerar os embeddings com um tamanho maior (1024), também aumentei o tamanho dos dados para 200.000 e o tamanho da camada oculta permaneceu em 512 e batch_size em 128.



#### **Random Forest**

In [37]:
# Caminhos dos arquivos
report_rf_path = "dados/classification_report_RF2.csv"
metrics_rf_path = "dados/metricsRF2.json"
#confusion_matrix_rf_path = "string_data/confusion_matrix_RF2.png"

# Carregar e imprimir os relatórios e métricas do Random Forest
load_and_print_classification_report(report_rf_path, "Random Forest")
load_and_print_metrics(metrics_rf_path, "Random Forest")
#load_and_display_image(confusion_matrix_rf_path, "Matriz de Confusão - Random Forest")


=== Relatório de Classificação - Random Forest ===
              precision    recall  f1-score       support
0              0.721053  0.756985  0.738582  20007.000000
1              0.744051  0.706947  0.725025  19993.000000
accuracy       0.731975  0.731975  0.731975      0.731975
macro avg      0.732552  0.731966  0.731804  40000.000000
weighted avg   0.732548  0.731975  0.731806  40000.000000

=== Métricas - Random Forest ===
accuracy: 0.7320
precision: 0.7441
recall: 0.7069
f1_score: 0.7250


O Random Forest apresentou uma melhoria  em relação aos resultados anteriores, principalmente na precisão.

A precisão e o recall estão equilibrados, indicando que o modelo está performando bem tanto para a classe positiva (interação) quanto para a classe negativa (não interação).

O F1-Score de 72.50% reforça que o modelo está generalizando bem para ambas as classes.

#### **LSTM**

A acurácia aumentou gradualmente ao longo das épocas, partindo de 59.57% na primeira época e estabilizando em torno de 67.02% a partir da 10ª época, o que é menor do que a obtida anteriormente. No entando, a precisão aumentou.

A precisão para a classe positiva (interação) é 77.80%, o que significa que, quando o modelo prevê uma interação, ele está frequentemente correto.

No entanto, o recall para a classe positiva é baixo (47.58%), indicando que o modelo está falhando em identificar muitas interações reais (falsos negativos).  Consequentemente, o F1-score também diminuiu.

Por conta dessa piora na acurácia e no recall, decidi não continuar nessa abordagem.

In [36]:
# Caminhos dos arquivos
report_lstm_path = "dados/classification_report_lstm_attention2.csv"
metrics_lstm_path = "dados/metrics_lstm_attention2.json"
#confusion_matrix_lstm_path = "string_data/confusion_matrix_lstm_attention2.png"

# Carregar e imprimir os relatórios e métricas do Random Forest
load_and_print_classification_report(report_lstm_path, "LSTM com Atenção")
load_and_print_metrics(metrics_lstm_path, "LSTM com Atenção")
#load_and_display_image(confusion_matrix_lstm_path, "Matriz de Confusão - LSTM com Atenção")


=== Relatório de Classificação - LSTM com Atenção ===
              precision    recall  f1-score      support
0.0            0.622655  0.864347  0.723859  20007.00000
1.0            0.778032  0.475817  0.590503  19993.00000
accuracy       0.670150  0.670150  0.670150      0.67015
macro avg      0.700344  0.670082  0.657181  40000.00000
weighted avg   0.700316  0.670150  0.657204  40000.00000

=== Métricas - LSTM com Atenção ===
accuracy: 0.6702
precision: 0.7780
recall: 0.4758
f1_score: 0.5905


##### **OBS:** Por falta de atenção, sobrescrevi as matrizes de confusão e acabei perdendo as imagens

## Discussão dos Resultados

**Desempenho dos Modelos**

Os modelos testados para prever interações proteína-proteína (PPI) apresentaram os seguintes resultados:

Random Forest obteve 73% de melhor acurácia.
LSTM alcançou 70% de acurácia após 50 épocas.

Ambos os modelos apresentam desempenho acima do acaso (50%), mas não ultrapassam em muito 70% de precisão e acurácia, indicando que ainda há espaço para melhorias.

Vários fatores podem ter contribuído para essa performance moderada:

* **Complexidade do Problema:** A predição de interações proteína-proteína (PPI) é um problema complexo e altamente dependente de características biológicas que podem não ser completamente capturadas apenas pelas sequências de aminoácidos. Apesar de os embeddings gerados pelo ESM-2 serem informativos, eles podem não ser suficientes para capturar todas as nuances das interações proteicas.

* **Tamanho do Embedding:** O uso de embeddings de tamanho 512 pode não ser suficiente para capturar toda a complexidade das sequências de proteínas. Embora o ESM-2 seja um modelo poderoso, a redução do tamanho do embedding pode ter limitado a capacidade do modelo de representar adequadamente as sequências.

* **Hiperparâmetros dos Modelos:** Apesar de terem sido testados vários hiperparâmetros, é possível que os modelos não tenham sido totalmente otimizados. No caso do Random Forest, o número de estimadores foi aumentado até 300, mas outros parâmetros como a profundidade das árvores ou o critério de divisão poderiam ser ajustados. Para a LSTM, embora tenham sido testadas várias combinações de hiperparâmetros, a arquitetura pode não ser a mais adequada para capturar dependências de longo prazo nas sequências de proteínas.

## Perspectivas Futuras

Para melhorar a performance dos modelos, várias abordagens podem ser consideradas:

* **Uso de Transformers para Classificação:** Em vez de usar embeddings pré-treinados e modelos clássicos como Random Forest ou LSTM, uma abordagem promissora seria utilizar modelos baseados em Transformers diretamente para a tarefa de classificação. Esses modelos podem ser fine-tunados para a tarefa específica de predição de interações proteína-proteína, o que pode melhorar a capacidade de capturar relações complexas entre as sequências de proteínas.
* **Incorporação de Informações Adicionais:** Além das sequências de aminoácidos, a inclusão de informações adicionais, como estruturas tridimensionais das proteínas, domínios funcionais, ou dados de expressão gênica, pode enriquecer o modelo e melhorar a predição de interações.

